In [ ]:
# Uncomment next line to recreate requirements.txt
# ! pip list --format=freeze > requirements.txt
# install packages
! pip install -r requirements.txt > log_install.txt

# Imports

In [ ]:
import io
import os
import shutil
import glob
import requests
import pickle
from zipfile import ZipFile
from tempfile import mkdtemp

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display

from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer

sns.set()

# Fonctions

In [ ]:
# Chargement du dataset csv depuis un zip en ligne
def load(url: str) -> pd.DataFrame:
    filename = 'data.csv'

    if not os.path.isfile(filename): 
        # Request zip on AWS
        print(f'load data from {url}')
        response = requests.get(url)

        # Unzip file
        temp_dir = mkdtemp()
        print(f'extract to temp dir: {temp_dir}')

        with ZipFile(io.BytesIO(response.content), 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        # Copy file to current dir
        print(f'Copy csv to current directory')
        csv = glob.glob(os.path.join(temp_dir, '*.csv'))[0]
        shutil.copyfile(csv, filename)
        
        # Delete temp directory
        print('Delete temp Dir')
        shutil.rmtree(temp_dir, ignore_errors=True)
    else:
        print(f'{filename} already exist! Using it to load dataframe. (Delete it to reload from AWS S3 repository)')


    # Read it in pandas
    print('Read csv by pandas')
    df = pd.read_csv(filename, delimiter='\t',
                     parse_dates=True,
                     low_memory=False)

    print('Dataset ready to explore')
    return df

# Affichage des infos du dataset
def infos(df: pd.DataFrame):
    memory_gb = np.round(df.memory_usage(deep=True).sum()/(1024**3),2)
    nb_lignes = df.shape[0]
    nb_columns = df.shape[1]
    print(f'A ce stade le dataset contient {nb_lignes} lignes et {nb_columns} colonnes. (conso mémoire {memory_gb}Gb)')
    
# Graphe des taux de remplissage
def remplissage(df: pd.DataFrame):
    df_na = pd.DataFrame(((1- df.isna().mean()) * 100).round(0)).rename(columns={0: 'mean'})
    ax = sns.barplot(data=df_na, x='mean', y=df_na.index, palette='ocean', order=df_na.sort_values(by='mean', ascending=False).index)
    ax.bar_label(ax.containers[0], fmt='%.0f%%', padding=-35, c="white")
    plt.title('Taux de remplissage des colonnes')


# Extraction d'une serie depuis un champ contenant une liste
def extract_serie(dataFrame: pd.DataFrame, column: str) -> pd.Series:
    words = pd.Series(dtype=np.int32)
    for value in dataFrame.loc[:, column].dropna():
        if type(value) == str:
            splitted_values = value.split(',')
            for splitted_value in splitted_values:
                if splitted_value in words.keys():
                    words[splitted_value] += 1
                else:
                    words[splitted_value] = 0
    
    return words.sort_values(ascending=False)        

# Scaler de données
def scale(values: np.ndarray) -> np.ndarray:
    scaler = StandardScaler()
    scaled_values = scaler.fit_transform(values)
    pd.DataFrame(scaled_values).describe().round(2)
    return scaled_values

# Acp
def acp(scaled_values: np.ndarray, n_components = 2) -> {'pca': PCA, 'X_proj': np.ndarray}:
    pca = PCA(n_components=n_components)
    pca.fit(scaled_values)
    return {'pca': pca, 'X_proj': pca.transform(scaled_values)}

# Affichage de 'éboulie des valeurs propres
def acp_eboulis(pca: PCA):
    scree = (pca.explained_variance_ratio_*100).round(2)
    scree_cum = scree.cumsum()
    x = range(1, len(scree)+1)
    plt.bar(height=scree, x=x)
    plt.plot(x, scree_cum, '-o', c='r')
    plt.title('Eboulis des valeurs propres')    
    
# Cercle des correlations
def correlation_graph(pca, 
                      x_y, 
                      features) : 
    """Affiche le graphe des correlations

    Positional arguments : 
    -----------------------------------
    pca : sklearn.decomposition.PCA : notre objet PCA qui a été fit
    x_y : list ou tuple : le couple x,y des plans à afficher, exemple [0,1] pour F1, F2
    features : list ou tuple : la liste des features (ie des dimensions) à représenter
    """

    # Extrait x et y 
    x,y=x_y

    # Taille de l'image (en inches)
    fig, ax = plt.subplots(figsize=(10, 9))

    # Pour chaque composante : 
    for i in range(0, pca.components_.shape[1]):

        # Les flèches
        ax.arrow(0,0, 
                pca.components_[x, i],  
                pca.components_[y, i],  
                head_width=0.07,
                head_length=0.07, 
                width=0.02, )

        # Les labels
        plt.text(pca.components_[x, i] + 0.05,
                pca.components_[y, i] + 0.05,
                features[i])
        
    # Affichage des lignes horizontales et verticales
    plt.plot([-1, 1], [0, 0], color='grey', ls='--')
    plt.plot([0, 0], [-1, 1], color='grey', ls='--')

    # Nom des axes, avec le pourcentage d'inertie expliqué
    plt.xlabel('F{} ({}%)'.format(x+1, round(100*pca.explained_variance_ratio_[x],1)))
    plt.ylabel('F{} ({}%)'.format(y+1, round(100*pca.explained_variance_ratio_[y],1)))
    plt.title("Cercle des corrélations (F{} et F{})".format(x+1, y+1))

    # Le cercle 
    an = np.linspace(0, 2 * np.pi, 100)
    plt.plot(np.cos(an), np.sin(an))  # Add a unit circle for scale

    # Axes et display
    plt.axis('equal')
    plt.show(block=False)
    

# Affichage des point projetés de l'acp
def display_factorial_planes(   X_projected, 
                                x_y, 
                                pca=None, 
                                labels = None,
                                clusters=None, 
                                alpha=1,
                                figsize=[10,8], 
                                marker="." ):
    """
    Affiche la projection des individus

    Positional arguments : 
    -------------------------------------
    X_projected : np.array, pd.DataFrame, list of list : la matrice des points projetés
    x_y : list ou tuple : le couple x,y des plans à afficher, exemple [0,1] pour F1, F2

    Optional arguments : 
    -------------------------------------
    pca : sklearn.decomposition.PCA : un objet PCA qui a été fit, cela nous permettra d'afficher la variance de chaque composante, default = None
    labels : list ou tuple : les labels des individus à projeter, default = None
    clusters : list ou tuple : la liste des clusters auquel appartient chaque individu, default = None
    alpha : float in [0,1] : paramètre de transparence, 0=100% transparent, 1=0% transparent, default = 1
    figsize : list ou tuple : couple width, height qui définit la taille de la figure en inches, default = [10,8] 
    marker : str : le type de marker utilisé pour représenter les individus, points croix etc etc, default = "."
    """

    # Transforme X_projected en np.array
    X_ = np.array(X_projected)

    # On définit la forme de la figure si elle n'a pas été donnée
    if not figsize: 
        figsize = (7,6)

    # On gère les labels
    if  labels is None : 
        labels = []
    try : 
        len(labels)
    except Exception as e : 
        raise e

    # On vérifie la variable axis 
    if not len(x_y) ==2 : 
        raise AttributeError("2 axes sont demandées")   
    if max(x_y )>= X_.shape[1] : 
        raise AttributeError("la variable axis n'est pas bonne")   

    # on définit x et y 
    x, y = x_y

    # Initialisation de la figure       
    fig, ax = plt.subplots(1, 1, figsize=figsize)

    # On vérifie s'il y a des clusters ou non
    c = None if clusters is None else clusters
 
    # Les points    
    # plt.scatter(   X_[:, x], X_[:, y], alpha=alpha, 
    #                     c=c, cmap="Set1", marker=marker)
    sns.scatterplot(data=None, x=X_[:, x], y=X_[:, y], hue=c)

    # Si la variable pca a été fournie, on peut calculer le % de variance de chaque axe 
    if pca : 
        v1 = str(round(100*pca.explained_variance_ratio_[x]))  + " %"
        v2 = str(round(100*pca.explained_variance_ratio_[y]))  + " %"
    else : 
        v1=v2= ''

    # Nom des axes, avec le pourcentage d'inertie expliqué
    ax.set_xlabel(f'F{x+1} {v1}')
    ax.set_ylabel(f'F{y+1} {v2}')

    # Valeur x max et y max
    x_max = np.abs(X_[:, x]).max() *1.1
    y_max = np.abs(X_[:, y]).max() *1.1

    # On borne x et y 
    ax.set_xlim(left=-x_max, right=x_max)
    ax.set_ylim(bottom= -y_max, top=y_max)

    # Affichage des lignes horizontales et verticales
    plt.plot([-x_max, x_max], [0, 0], color='grey', alpha=0.8)
    plt.plot([0,0], [-y_max, y_max], color='grey', alpha=0.8)

    # Affichage des labels des points
    if len(labels) : 
        for i,(_x,_y) in enumerate(X_[:,[x,y]]):
            try:
                plt.text(_x, _y+0.05, labels[i], fontsize='14', ha='center',va='center', family=['DejaVu Sans', 'Segoe UI'],)
            except:
                print("An exception occurred")

    # Titre et display
    plt.title(f"Projection des individus (sur F{x+1} et F{y+1})")
    plt.show()



# Application

L'idée de l'application est de fournir aux agents de Santé Publique France la possibilité de visualiser graphiquement la composition energetique d'une catégorie d'aliments
L'énergie se calcul grâce à la quantité de proteines/Lipides/Glucides: https://www.nutrisens.com/vitalites/comment-decrypter-les-valeurs-nutritionnelles/

A savoir que:
- Les lipides (Graisses) se nomme <b>"fat"</b> en anglais
- Les glucides (Sucres) sont aussi nommées <b>"carbohydrates"</b> scientifiquement parlant

Hypothèses à vérifier dans ce dataset:

1. On peut extraire la liste des catégories dans lesquels sont déclarés les produits
3. On va pouvoir extrapoler les données manquantes d'une des composantes en connaissant les autres
4. On peut déduire des données que l'energie est correlée aux macronutriments (proteines, lipides glucides)

## Lecture et apperçu du dataset

In [ ]:
url = 'https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/fr.openfoodfacts.org.products.csv.zip'    
df = load(url)
df_original = df.copy()

In [ ]:
df.head()

In [ ]:
infos(df)

## Filtrage des colonnes necessaires à l'étude

Pour ce projet je garde le nom des produits, les macro-nutriments, energie et nutriscore/nutrigrade

In [ ]:
df = df.loc[:, ['product_name','energy_100g', 'proteins_100g', 'carbohydrates_100g', 'fat_100g', 'nutrition-score-fr_100g', 'nutrition_grade_fr', 'fiber_100g', 'categories_fr']]

In [ ]:
df.describe()

In [ ]:
remplissage(df)

## Nettoyage

### Suppressions des valeurs abérantes

Mise à NaN des macro-nutriments dont des valeurs sont > 100 et < 0

In [ ]:
df.loc[df['proteins_100g'] < 0, 'proteins_100g'] = np.nan
df.loc[df['carbohydrates_100g'] < 0, 'carbohydrates_100g'] = np.nan
df.loc[df['fat_100g'] < 0, 'fat_100g'] = np.nan

df.loc[df['proteins_100g'] > 100, 'proteins_100g'] = np.nan
df.loc[df['carbohydrates_100g'] > 100, 'carbohydrates_100g'] = np.nan
df.loc[df['fat_100g'] > 100, 'fat_100g'] = np.nan

### Suppression des lignes vides

In [ ]:
nb_lignes_vides = df.loc[df.isna().sum(axis=1) == df.shape[1], :]
print(f'Suppression des {len(nb_lignes_vides)} lignes vides')
df = df.loc[df.isna().sum(axis=1) != df.shape[1], :]
infos(df)

### Extaction d'un subset permettant l'imputation de valeurs manquantes par regression lineaire

L'energie étant proportionnelle aux proteines, glucides et lipides, on devrait pouvoir deduire les valeurs de chacunes d'elle en fonction de l'energie déclarée

Pour consolider cette hypothese, on va recalculer une energie théorique et extraire un subset qui suit les règles suivantes:
- au moins une donnée parmis proteine/glucide/lipide est présente et inclue dans les quantiles 0.25/0.75
- L'energie déclarée est comprise entre 0 et 3800 pour 100g (3800Kcal etant le maximum théorique possible)
- L'energie déclarée est à +- 0.5% de l'energie recalculée

visualisation des qantiles

In [ ]:
sns.boxplot(data = df.loc[:, ['proteins_100g', 'carbohydrates_100g', 'fat_100g', 'fiber_100g']], showfliers=False, orient='h')
plt.title('Distribution des macro-nutriments')
plt.show()

Extraction du subset suivant les règles énoncées

In [ ]:
sample = df.drop('categories_fr', axis=1).dropna()

proteins = sample['proteins_100g']
proteins_inliers = proteins.between(proteins.quantile(0.25), proteins.quantile(0.75))

carbohydrates = sample['carbohydrates_100g']
carbohydrates_inliers = carbohydrates.between(carbohydrates.quantile(0.25), carbohydrates.quantile(0.75))

fat = sample['fat_100g']
fat_inliers = fat.between(fat.quantile(0.25), fat.quantile(0.75))

sample['calc_energy'] = sample['proteins_100g'] * 17 + sample['fat_100g'] * 38 + sample['carbohydrates_100g'] * 17

cond1 = (sample['proteins_100g'] > 0) | (sample['carbohydrates_100g'] > 0) | (sample['fat_100g'] > 0)
cond2 = (sample['energy_100g'] >= 0) & (sample['energy_100g'] <= 3800)
cond3 = proteins_inliers & carbohydrates_inliers & fat_inliers
cond4 = sample['energy_100g'].between(sample['calc_energy']*0.995, sample['calc_energy']*1.005)
sample = sample.loc[cond1 & cond2 & cond3 & cond4, :]
sample.head()

In [ ]:
sample.shape

### Visualisation des corrélations des données du subset

In [ ]:
g = sns.pairplot(sample, hue='nutrition_grade_fr', hue_order=['a', 'b', 'c', 'd', 'e'], corner=True,  diag_kws=dict(fill=False))
g.fig.suptitle('Relations bivariées des differentes caractéristiques')
plt.show()

In [ ]:
correlations = sample.corr(numeric_only=True)
figure = plt.figure(figsize=(6,6))
sns.heatmap(correlations, square=True, linewidths=0.01, cmap="coolwarm")
plt.title('Correlation entre les differentes caractéristiques')
plt.show()

### Constat

Malgré une amplitude importante, l'energie a une correlation lineaire avec les lipides et le carbohydrate.
On va utiliser ces corrélations pour remplir les valeurs manquantes lorsque l'energie est déclarée.

Le nutriscore est bien correlé au nutrigrade

### Imputation des lipides par regression linéaire entre lipides et energie

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(np.array(sample['fat_100g'].to_list()).reshape(-1, 1), np.array(sample['energy_100g'].to_list()).reshape(-1, 1))

In [ ]:
score = regr.score(np.array(sample['fat_100g'].to_list()).reshape(-1, 1), np.array(sample['energy_100g'].to_list()).reshape(-1, 1))
score = (score * 100).round(1)

In [ ]:
sns.scatterplot(data=sample, x='fat_100g', y='energy_100g')
plt.plot([0, 25], [regr.intercept_, regr.intercept_ + regr.coef_[0] * 25], '--', c='r')
plt.title(f'Rapport Lipides/Energie et courbe de regression (score = {score}%)')
plt.show

Le score de 64.3% n'est pas des meilleurs, mais je décide de garder cette méthode d'imputation

In [ ]:
fat_na = df['fat_100g'].isna() & ~df['energy_100g'].isna()
energy = df.loc[fat_na, 'energy_100g']
fat = np.clip((energy - regr.intercept_) / regr.coef_[0], 0, 100)
df.loc[fat_na, 'fat_100g'] = fat

### Imputation des carbohydrates par regression linéaire entre carbohydrates et energie

In [ ]:
regr2 = linear_model.LinearRegression()
regr2.fit(np.array(sample['carbohydrates_100g'].to_list()).reshape(-1, 1), np.array(sample['energy_100g'].to_list()).reshape(-1, 1))

In [ ]:
score = regr2.score(np.array(sample['carbohydrates_100g'].to_list()).reshape(-1, 1), np.array(sample['energy_100g'].to_list()).reshape(-1, 1))
score = (score * 100).round(1)

In [ ]:
sns.scatterplot(data=sample, x='carbohydrates_100g', y='energy_100g')
plt.plot([0, 65], [regr2.intercept_, regr2.intercept_ + regr2.coef_[0] * 65], '--', c='r')
plt.title(f'Rapport Carbohydrates/Energie et courbe de regression (score = {score}%)')
plt.show

Le score de 75.3% est meilleur que pour les lipides.

Imputation si valeur energie présente:

In [ ]:
ch_na = df['carbohydrates_100g'].isna() & ~df['energy_100g'].isna()
energy = df.loc[ch_na, 'energy_100g']
ch = np.clip((energy - regr2.intercept_) / regr2.coef_[0], 0, 100)
df.loc[ch_na, 'carbohydrates_100g'] = ch

### Bilan intermédiaire du taux de remplissage

In [ ]:
remplissage(df)

### Imputation des valeurs numériques manquantes restantes par KNNImputer

Je décide d'imputer les valeurs restantes grâce à un KNN imputer

In [ ]:
filename = 'X_imputed_knn'
numeric_cols = df.select_dtypes('number').columns
X = df.loc[:, numeric_cols].values

if not os.path.isfile(filename): 
    print('Knn execution, waiting please, and be happy!')
    imputer = KNNImputer(n_neighbors=5)
    X_imputed = imputer.fit_transform(X)
    pickle.dump(X_imputed, open(filename, 'wb'))
else:
    print(f'Knn was already executed. be happy! (if dataset has changed please remove file {filename} and reexecute cell)')

X_imputed = pickle.load(open(filename, 'rb'))    
df[numeric_cols] = pd.DataFrame(X_imputed, columns=numeric_cols, index=df.index)[numeric_cols]

Vérifications si des valeurs abérantes ont été imputées par le KNN

In [ ]:
sns.boxplot(data = df.loc[:, ['proteins_100g', 'carbohydrates_100g', 'fat_100g', 'fiber_100g']], showfliers=False, orient='h')
plt.title('Distribution des macro-nutriments')
plt.show()

### affectation des nutrigrades manquants

informations ici: https://nutriscore.impag.ch

In [ ]:
df.loc[df['nutrition_grade_fr'].isna() & (df['nutrition-score-fr_100g'] < 0), 'nutrition_grade_fr'] = 'a'
df.loc[df['nutrition_grade_fr'].isna() & (df['nutrition-score-fr_100g'] >= 0) & (df['nutrition-score-fr_100g'] <= 2), 'nutrition_grade_fr'] = 'b'
df.loc[df['nutrition_grade_fr'].isna() & (df['nutrition-score-fr_100g'] > 2) & (df['nutrition-score-fr_100g'] <= 10), 'nutrition_grade_fr'] = 'c'
df.loc[df['nutrition_grade_fr'].isna() & (df['nutrition-score-fr_100g'] > 10) & (df['nutrition-score-fr_100g'] <= 18), 'nutrition_grade_fr'] = 'd'
df.loc[df['nutrition_grade_fr'].isna() & (df['nutrition-score-fr_100g'] > 18), 'nutrition_grade_fr'] = 'e'

In [ ]:
remplissage(df)

In [ ]:
infos(df)

### Suppression des lignes dupliquées

In [ ]:
print(f'Suppression de {df.duplicated().sum()} lignes duppliquées')
df.drop_duplicates(inplace=True)
infos(df)

Fin du nettoyage des données

on peut donc imaginer extraire les catégories et proceder à des comparaisons dans la future application

## Exploration des données

### Répartition des macro-nutriments par nutrigrade

In [ ]:
sns.boxplot(data=df, x='proteins_100g', y='nutrition_grade_fr', orient='h', order=['a','b','c','d','e'], showfliers=False)
plt.title('Répartition des proteïnes par nutrigrade')
plt.show()

sns.boxplot(data=df, x='carbohydrates_100g', y='nutrition_grade_fr', orient='h', order=['a','b','c','d','e'], showfliers=False)
plt.title('Répartition des carbohydrates par nutrigrade')
plt.show()

sns.boxplot(data=df, x='fat_100g', y='nutrition_grade_fr', orient='h', order=['a','b','c','d','e'], showfliers=False)
plt.title('Répartition des lipides par nutrigrade')
plt.show()

Plus les lipides et glucides sont importants plus le nutrigrade est élévé

### Analyse des composantes principales

In [ ]:
values = df[numeric_cols].values
scaled_values = scale(values)

In [ ]:
scaled_df = pd.DataFrame(scaled_values, columns=numeric_cols)
scaled_df.describe().round(2).loc[['mean', 'std'], :]

In [ ]:
result = acp(scaled_values=scaled_values, n_components=6)

In [ ]:
result['pca'].explained_variance_ratio_

In [ ]:
acp_eboulis(pca=result['pca'])

In [ ]:
correlation_graph(result['pca'], (0,1), numeric_cols)

Axe 1: On constate que le nutriscore et les lipides composent ce axe. Il est le reflet de la politique du nutriscore qui consiste à préférer exclure les lipides de l'alimentation. 

Axe 2: C'est la composante des glucides. Elle n'a quasiment pas d'influence sur le nutriscore. Et l'on constate que les proteines sont anti corrélées à cet axe. Un produit est donc plutôt soit proteïnique soit glucidique, mais difficilement les 2 ensembles

In [ ]:
correlation_graph(result['pca'], (2,3), numeric_cols)

In [ ]:
display_factorial_planes(result['X_proj'], (0,1), result['pca'])

### Analyse des categories

In [ ]:
df.loc[:, 'categories_fr'].dropna().unique()[:10]

Certaine valeurs apparaissent plusieurs fois, je crée et utilise une fonction generique qui renverra une serie basée sur une colonne avec leur nombre d'apparitions

Je garde celles qui concernent au moins 1000 produits

In [ ]:
categories = extract_serie(df, 'categories_fr')

In [ ]:
categories_subset = categories.loc[categories >=1000]

In [ ]:
plt.figure(figsize=(10,14))
sns.barplot(x=categories_subset.values, y=categories_subset.index, orient='h')
plt.title('Nombre de produits par catégorie')
plt.show()

## Composition energetique d'une catégorie d'aliments

In [ ]:
dd1 = widgets.Dropdown(options=categories_subset.index.to_list(), value=categories_subset.index.to_list()[0], description='Categorie')
dd2 = widgets.SelectMultiple(options={'Energie': 'energy_100g', 'Proteïnes': 'proteins_100g', 'Lipides': 'fat_100g', 'Glucides': 'carbohydrates_100g', 'Fibres': 'fiber_100g', 'Nutriscore': 'nutrition-score-fr_100g'}, value=['energy_100g'], description='Valeur observée')
dd3 = widgets.IntSlider(min=5, max=20, step=1, description='Découpage')
ui = widgets.HBox([dd1, dd2, dd3])

def draw(categorie, x, bins):
    plt.figure(figsize=(10,5))
    data = df.loc[(~df['categories_fr'].isna()) & (df['energy_100g'] <= 3800), :]
    data = data.loc[data['categories_fr'].str.contains(categorie), x]
    sns.histplot(data=data, bins=bins, element="step")
    plt.title(f'Répartition de la valeur "{x}" pour la catégorie "{categorie}"')
    plt.show()
    
out = widgets.interactive_output(draw, {'categorie': dd1, 'x':dd2, 'bins':dd3})
display(ui, out)